In [1]:
import findspark
findspark.init()
import pyspark

In [2]:
pyspark.__file__

'C:\\spark\\python\\pyspark\\__init__.py'

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [11]:
import wget

In [54]:
wget.download('https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-02.parquet')

'fhvhv_tripdata_2021-02.parquet'

In [55]:
df = spark.read \
    .option("header", "true") \
    .parquet('fhvhv_tripdata_2021-02.parquet')

In [56]:
df.show()

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+--

In [26]:
from pyspark.sql.functions import date_format

In [62]:
df.withColumn("month", date_format("request_datetime", "MM")). \
    withColumn("day", date_format("request_datetime", "dd")). \
    show()

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+-----+---+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|month|day|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+---------------

In [67]:
df_assignment = df.withColumn("month", date_format("request_datetime", "MM")). \
    withColumn("day", date_format("request_datetime", "dd"))

In [68]:
df_assignment.show()

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+-----+---+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|month|day|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+---------------

In [69]:
print(df_assignment.dtypes)

[('hvfhs_license_num', 'string'), ('dispatching_base_num', 'string'), ('originating_base_num', 'string'), ('request_datetime', 'timestamp'), ('on_scene_datetime', 'timestamp'), ('pickup_datetime', 'timestamp'), ('dropoff_datetime', 'timestamp'), ('PULocationID', 'bigint'), ('DOLocationID', 'bigint'), ('trip_miles', 'double'), ('trip_time', 'bigint'), ('base_passenger_fare', 'double'), ('tolls', 'double'), ('bcf', 'double'), ('sales_tax', 'double'), ('congestion_surcharge', 'double'), ('airport_fee', 'double'), ('tips', 'double'), ('driver_pay', 'double'), ('shared_request_flag', 'string'), ('shared_match_flag', 'string'), ('access_a_ride_flag', 'string'), ('wav_request_flag', 'string'), ('wav_match_flag', 'string'), ('month', 'string'), ('day', 'string')]


In [70]:
df_assignment.registerTempTable('df_month')

C:\spark\python\pyspark\sql\dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [95]:
spark.sql("""
SELECT
    count(base_passenger_fare)
FROM
    df_month
WHERE
    base_passenger_fare IS NULL
OR
    base_passenger_fare == 0
""").show()

+--------------------------+
|count(base_passenger_fare)|
+--------------------------+
|                     18166|
+--------------------------+



There are 18166 cancelled order in the data

In [99]:
spark.sql("""
SELECT
    COUNT(day)
FROM
    df_month
WHERE
    base_passenger_fare IS NOT NULL
AND
    base_passenger_fare > 0
AND
    day == "15"
""").show()

+----------+
|count(day)|
+----------+
|    418263|
+----------+



Question 1: How many taxi trips were there on February 15? <br>
There is 418.263 trips on February 15

In [102]:
spark.sql("""
SELECT
    day, MAX(trip_miles)
FROM
    df_month
WHERE
    base_passenger_fare IS NOT NULL
AND
    base_passenger_fare > 0 
GROUP BY
    day
ORDER BY 
    day
""").show(n=29, truncate=False)

+---+---------------+
|day|max(trip_miles)|
+---+---------------+
|01 |212.43         |
|02 |282.78         |
|03 |184.26         |
|04 |203.97         |
|05 |245.35         |
|06 |275.32         |
|07 |216.36         |
|08 |253.5          |
|09 |480.73         |
|10 |512.5          |
|11 |240.66         |
|12 |250.11         |
|13 |226.24         |
|14 |207.44         |
|15 |173.582        |
|16 |307.661        |
|17 |254.37         |
|18 |187.96         |
|19 |224.33         |
|20 |329.16         |
|21 |340.64         |
|22 |196.59         |
|23 |347.41         |
|24 |301.73         |
|25 |190.272        |
|26 |192.465        |
|27 |232.15         |
|28 |454.49         |
+---+---------------+



Question 2: Find the longest trip for each day ? <br>
The Answer is on top of this markdown

In [109]:
spark.sql("""
SELECT
    dispatching_base_num, COUNT(dispatching_base_num) as most_frequent
FROM
    df_month
WHERE
    base_passenger_fare IS NOT NULL
AND
    base_passenger_fare > 0 
GROUP BY
    dispatching_base_num
ORDER BY 
    most_frequent DESC
LIMIT 5
""").show(truncate=False)

+--------------------+-------------+
|dispatching_base_num|most_frequent|
+--------------------+-------------+
|B02510              |3222621      |
|B02764              |964044       |
|B02872              |881381       |
|B02875              |684117       |
|B02765              |558815       |
+--------------------+-------------+



Question 3: Find Top 5 Most frequent `dispatching_base_num` ? <br>
The most frequent `dispatching_base_num` are B02510, B02764, B02872, B02875, and B02765

In [124]:
spark.sql("""
SELECT
    PULocationID, DOLocationID, COUNT(PULocationID) as pu_frequent, COUNT(DOLocationID) AS du_frequent
FROM
    df_month
WHERE
    base_passenger_fare IS NOT NULL
AND
    base_passenger_fare > 0    
GROUP BY 
    PULocationID, DOLocationID
ORDER BY
    PULocationID, DOLocationID DESC
LIMIT 5
""").show()

+------------+------------+-----------+-----------+
|PULocationID|DOLocationID|pu_frequent|du_frequent|
+------------+------------+-----------+-----------+
|           1|         263|         12|         12|
|           1|         262|          5|          5|
|           1|         261|          1|          1|
|           1|         260|          1|          1|
|           1|         256|          3|          3|
+------------+------------+-----------+-----------+



Question 4: Find Top 5 Most common location pairs (PUlocationID and DOlocationID) ?<br>
Most frequent PULocationID is 1 and DOLocationID are 263, 262, 261, 260, and 256